In [1]:
import pandas as pd
import numpy as np

positive_sample = pd.read_csv(r'data\samples\positive_sample.csv', nrows = 20000)
negative_sample = pd.read_csv(r'data\samples\negative_sample.csv', nrows = 20000)

# Create lists of reviews and target sentiment values
pos_rev = positive_sample['review'].astype(str).to_list()
pos_trg = positive_sample['sentiment'].to_list()

neg_rev = negative_sample['review'].astype(str).to_list()
neg_trg = negative_sample['sentiment'].to_list()


# Concatenate review lists and target lists
reviews = pos_rev + neg_rev
targets = pos_trg + neg_trg

Data Leakage in notebook 7! Split first then vectorize

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(reviews, targets)

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
X_train_f = vectorizer.fit_transform(X_train)
X_test_f = vectorizer.transform(X_test)

clf = LogisticRegression(max_iter=200).fit(X_train_f, y_train)
y_pred = clf.predict(X_test_f)

print(accuracy_score(y_test,y_pred))

0.8686


Accuracy still around 87%, likely because of large max features threshold. 

In [3]:
test = 'i love this game'
test_transf = vectorizer.transform([test])
print(clf.predict(test_transf))

[ True]


In [4]:
# Overwrite old features and coefficients with leakage free values
feature_names = vectorizer.get_feature_names()
view = pd.DataFrame({'features':feature_names, 'coeff': clf.coef_[0] })
view.to_csv(r'data\visualization\raw data\7-tfidsample.csv', index=False)

Testing out Decision Tree classifier

In [5]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier().fit(X_train_f,y_train)
y_dt = dt_clf.predict(X_test_f)
print(accuracy_score(y_test,y_dt))

0.7614


Worse Performance. Likely because of binary nature of decision trees with the variability of sentence length and number of word occurences. Random Forest could perform better.

In [6]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=50).fit(X_train_f, y_train)
y_rf = rf_clf.predict(X_test_f)
print(accuracy_score(y_test,y_rf))

0.8227


Slightly better, but binary classifiers still naturally incompatible with this problem. Logistic regression performance is acceptable.